In [2]:
import os
import random
import shutil

def split_dataset(source_dir, train_dir, test_dir, split_ratio=0.8, seed=None):
    if seed is not None:
        random.seed(seed)
    
    # 경로 지정
    source_dir = r"C:\Users\angel\Desktop\flower_photos"
    train_dir = r"C:\Users\angel\Desktop\flower_train_test\train"
    test_dir = r"C:\Users\angel\Desktop\flower_train_test\test"
    
    # 타겟 디렉토리 생성
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    # 꽃 종류별 하위 폴더 리스트를 가져옵니다.
    flower_types = os.listdir(source_dir)

    for flower_type in flower_types:
        # 각 꽃 종류별 하위 폴더의 파일 리스트를 가져옵니다.
        flower_folder = os.path.join(source_dir, flower_type)
        file_list = os.listdir(flower_folder)
        num_files = len(file_list)
        num_train_files = int(num_files * split_ratio)

        # 파일 리스트를 섞은 후 train과 test로 분류합니다.
        random.shuffle(file_list)
        train_files = file_list[:num_train_files]
        test_files = file_list[num_train_files:]

        # train_files를 train 디렉토리로 복사합니다.
        for file in train_files:
            src_path = os.path.join(flower_folder, file)
            dest_path = os.path.join(train_dir, flower_type, file)
            os.makedirs(os.path.dirname(dest_path), exist_ok=True)
            shutil.copy(src_path, dest_path)

        # test_files를 test 디렉토리로 복사합니다.
        for file in test_files:
            src_path = os.path.join(flower_folder, file)
            dest_path = os.path.join(test_dir, flower_type, file)
            os.makedirs(os.path.dirname(dest_path), exist_ok=True)
            shutil.copy(src_path, dest_path)

source_dir = r"C:\Users\angel\Desktop\flower_photos"
train_dir = r"C:\Users\angel\Desktop\flower_train_test\train"
test_dir = r"C:\Users\angel\Desktop\flower_train_test\test"
split_ratio = 0.8        # train 데이터셋의 비율

split_dataset(source_dir, train_dir, test_dir, split_ratio)

In [1]:
import tensorflow as tf
from keras import layers, models

def VGGNet():
    model = models.Sequential()

    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(4096, activation='relu'))
    model.add(layers.Dense(4096, activation='relu'))
    model.add(layers.Dense(5, activation='softmax')) # Assuming 1000 classes in the dataset

    return model

In [2]:
# 데이터 경로 설정
data_dir = r"C:\Users\angel\Desktop\fl ower_photos"
train_dir = r"C:\Users\angel\Desktop\flower_train_test\train"
test_dir = r"C:\Users\angel\Desktop\flower_train_test\test"

In [3]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

# 이미지 데이터를 불러오기 위한 ImageDataGenerator 설정
datagen = ImageDataGenerator(rescale=1./255)
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 2934 images belonging to 5 classes.


In [4]:
# VGGNet 모델 생성
model = VGGNet()
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [5]:
# 모델 학습
model.fit(train_generator, steps_per_epoch=40, epochs=10)

Epoch 1/10
40/40 [==============================] - 831s 21s/step - loss: 1.9289 - accuracy: 0.2344
Epoch 2/10
34/40 [========================>.....] - ETA: 1:58 - loss: 1.6041 - accuracy: 0.2335

In [3]:


# 테스트 데이터셋 불러오기
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False  # 테스트 데이터셋은 섞지 않도록 설정
)

# 모델 평가
loss, accuracy = model.evaluate(test_generator)
print(f"테스트 데이터셋 손실: {loss:.4f}, 정확도: {accuracy:.4f}")

Found 2934 images belonging to 5 classes.
Epoch 1/10
12/92 [==>...........................] - ETA: 29:17 - loss: 1.6703 - accuracy: 0.2031

KeyboardInterrupt: 